In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import warnings
import statsmodels.api as sm
from sklearn.decomposition import PCA

data = pd.read_csv("./test_sample.csv")
data.describe()

X = data.iloc[:,1:]
Y = data['Y']

In [2]:
def find_best_model(X, Y, max_vars=491, min_r_squared=0.9):
    for j in range(2, max_vars+1):
        X_j = X.iloc[:, :j]
        X_j = sm.add_constant(X_j)
        model = sm.OLS(Y, X_j).fit()
        r_squared = model.rsquared

        if r_squared > min_r_squared:
            return j

    return None
n_orig = find_best_model(X,Y)
print("smallest number R^2 > 0.9:",n_orig)

smallest number R^2 > 0.9: 341


In [3]:
def pca_analysis(X, n_components=491):
    pca = PCA(n_components=n_components)
    pca.fit(X)
    
    importance = pd.DataFrame({
        'Standard deviation': np.sqrt(pca.explained_variance_),
        'Proportion of Variance': pca.explained_variance_ratio_,
        'Cumulative Proportion': np.cumsum(pca.explained_variance_ratio_)
    }, index=[f"PC{i+1}" for i in range(n_components)])

    scores = pd.DataFrame(
        pca.transform(X),
        columns=[f"PC{i+1}" for i in range(n_components)]
    )   
    return importance, scores

xPCA_importance, factorScores=pca_analysis(X,491)

In [4]:
def rel_imp_me(X, y):
    names = X.columns
    ser = pd.Series(index=names)
    lm0 = sm.OLS(y, sm.add_constant(X)).fit()

    for c in names:
        lm = sm.OLS(y, sm.add_constant(X[names.drop(c)])).fit()
        ser[c] = lm0.rsquared - lm.rsquared

    res = pd.DataFrame(columns=['last', 'first', 'betasq', 'pratt'], index=names)
    res['last'] = ser
    corr = X.apply(lambda x: np.corrcoef(y, x)[0, 1], axis=0)
    res['first'] = corr ** 2
    sx = X.std()
    res['betasq'] = (lm0.params[names] * sx / np.std(y)) ** 2
    res['pratt'] = (lm0.params[names] * sx / np.std(y)) * corr
    return res
metrics_PCA = rel_imp_me(factorScores, Y)
print(metrics_PCA)

first_PCA_rank = metrics_PCA["first"].rank(ascending=False, method='first')
print(first_PCA_rank)


               last         first        betasq         pratt
PC1    1.092959e-03  1.092959e-03  1.095150e-03  1.094054e-03
PC2    1.855047e-02  1.855047e-02  1.858764e-02  1.856904e-02
PC3    1.112873e-02  1.112873e-02  1.115104e-02  1.113988e-02
PC4    5.137558e-03  5.137558e-03  5.147853e-03  5.142703e-03
PC5    6.401958e-03  6.401958e-03  6.414788e-03  6.408370e-03
...             ...           ...           ...           ...
PC487  5.159097e-07  5.159097e-07  5.169436e-07  5.164264e-07
PC488  5.460891e-06  5.460891e-06  5.471835e-06  5.466361e-06
PC489  3.359578e-06  3.359578e-06  3.366311e-06  3.362943e-06
PC490  2.518500e-09  2.518500e-09  2.523547e-09  2.521022e-09
PC491  7.191697e-08  7.191697e-08  7.206109e-08  7.198899e-08

[491 rows x 4 columns]
PC1      173.0
PC2        9.0
PC3       18.0
PC4       57.0
PC5       40.0
         ...  
PC487    480.0
PC488    442.0
PC489    457.0
PC490    491.0
PC491    487.0
Name: first, Length: 491, dtype: float64


In [5]:
metrics_PCA_sort = pd.DataFrame({"Factors" : first_PCA_rank.index,
                                 "Rank" : first_PCA_rank.values}).sort_values(by="Rank") 
orderedFactors = pd.DataFrame(factorScores, columns= metrics_PCA_sort["Factors"])

In [6]:
def rSquar(j, y, X) :
    return sm.OLS(y, sm.add_constant(X.iloc[:,:j])).fit().rsquared

In [7]:
target_r_squared = 0.9 

n_PCA = None
model_dimensionality_reduction = None

orderedPCA_R2 = [rSquar(j,Y, orderedFactors) for j in range(2,492)]

for j in range(2, 492):
    if orderedPCA_R2[j - 2] >= target_r_squared:
        n_PCA = j
        model_dimensionality_reduction = n_orig - n_PCA
        print("Determination coefficient(R^2):", orderedPCA_R2[j - 2])
        break

print("Smallest number of PCA factors:", n_PCA)
print("Model dimensionality reduction:", model_dimensionality_reduction)


Determination coefficient(R^2): 0.9008394770655794
Smallest number of PCA factors: 157
Model dimensionality reduction: 184
